In [1]:
import cv2
import numpy as np
import pickle
import sys
from tensorflow.keras.models import load_model
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QVBoxLayout, QHBoxLayout, QWidget, QLabel, \
    QFileDialog
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer, Qt


class EmotionRecognitionApp(QMainWindow):
    def __init__(self):
        super().__init__()

        # Model ve etiket kodlayıcısını yükle
        self.model = load_model("emotion_model.h5")
        with open("label_encoder.pkl", "rb") as f:
            self.label_encoder = pickle.load(f)

        # Yüz tespiti için Cascade sınıflandırıcısını yükle
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        # Kamera ve diğer değişkenler
        self.cap = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)

        # Arayüz öğelerini oluştur
        self.init_ui()

        print("✅ Model ve label encoder başarıyla yüklendi!")

    def init_ui(self):
        # Ana pencere ayarları
        self.setWindowTitle("Duygu Tanıma Uygulaması")
        self.setGeometry(100, 100, 800, 600)

        # Ana widget ve düzen
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)

        # Görüntü etiketi
        self.image_label = QLabel()
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setMinimumSize(640, 480)
        main_layout.addWidget(self.image_label)

        # Durum etiketi
        self.status_label = QLabel("Hazır")
        self.status_label.setAlignment(Qt.AlignCenter)
        main_layout.addWidget(self.status_label)

        # Butonlar için düzen
        button_layout = QHBoxLayout()

        # Kamera butonu
        self.camera_button = QPushButton("Canlı Kamera")
        self.camera_button.clicked.connect(self.toggle_camera)
        button_layout.addWidget(self.camera_button)

        # Fotoğraf butonu
        self.photo_button = QPushButton("Fotoğraf Seç")
        self.photo_button.clicked.connect(self.open_photo)
        button_layout.addWidget(self.photo_button)

        main_layout.addLayout(button_layout)

    def toggle_camera(self):
        if self.timer.isActive():
            # Kamerayı durdur
            self.timer.stop()
            self.cap.release()
            self.cap = None
            self.camera_button.setText("Canlı Kamera")
            self.status_label.setText("Kamera durduruldu")
        else:
            # Kamerayı başlat
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                self.status_label.setText("Kamera açılamadı!")
                return

            self.timer.start(30)  # 30ms (yaklaşık 33 FPS)
            self.camera_button.setText("Kamerayı Durdur")
            self.status_label.setText("Kamera aktif")

    def open_photo(self):
        # Kamera aktifse durdur
        if self.timer.isActive():
            self.toggle_camera()

        # Dosya seçme penceresi
        file_path, _ = QFileDialog.getOpenFileName(self, "Fotoğraf Seç", "", "Image Files (*.png *.jpg *.jpeg *.bmp)")

        if file_path:
            # Fotoğrafı yükle ve analiz et
            self.status_label.setText(f"Fotoğraf analiz ediliyor: {file_path}")
            img = cv2.imread(file_path)
            analyzed_img = self.analyze_image(img)

            # Görüntüyü ekranda göster
            self.display_image(analyzed_img)
            self.status_label.setText("Fotoğraf analiz edildi")

    def update_frame(self):
        if self.cap is not None:
            ret, frame = self.cap.read()
            if ret:
                # Kare analiz et
                analyzed_frame = self.analyze_image(frame)

                # Görüntüyü ekranda göster
                self.display_image(analyzed_frame)

    def analyze_image(self, image):
        # Görüntü kopyası oluştur
        result_image = image.copy()

        # Gri tonlamalı görüntüye dönüştür
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Yüzleri tespit et
        faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            # Yüzün çerçevesini çiz
            cv2.rectangle(result_image, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Yüz bölgesini al ve yeniden boyutlandır
            face = gray[y:y + h, x:x + w]
            face_resized = cv2.resize(face, (48, 48))

            # Normalleştir ve model için hazırla
            face_normalized = face_resized.astype('float32') / 255.0
            face_normalized = np.reshape(face_normalized, (1, 48, 48, 1))

            # Tahmin yap
            prediction = self.model.predict(face_normalized)
            label = np.argmax(prediction)

            # Duygu etiketini al ve görüntüye yaz
            emotion = self.label_encoder.inverse_transform([label])[0]
            cv2.putText(result_image, emotion, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        return result_image

    def display_image(self, image):
        # OpenCV BGR -> RGB dönüşümü
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # QImage ve QPixmap oluştur
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)

        # Görüntüyü label'a koy
        pixmap = QPixmap.fromImage(qt_image)
        self.image_label.setPixmap(pixmap.scaled(self.image_label.width(),
                                                 self.image_label.height(),
                                                 Qt.KeepAspectRatio))

    def closeEvent(self, event):
        # Program kapatılırken kaynakları serbest bırak
        if self.cap is not None:
            self.cap.release()
        event.accept()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = EmotionRecognitionApp()
    window.show()
    sys.exit(app.exec_())


✅ Model ve label encoder başarıyla yüklendi!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/ste

SystemExit: 0

D:\ProgramData\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
